<a href="https://colab.research.google.com/github/Trung0Minh/AIO2023-MODULE-2/blob/main/text_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install datasets

In [35]:
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.13.1
    Uninstalling datasets-2.13.1:
      Successfully uninstalled datasets-2.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

dataset = load_dataset("microsoft/ms_marco", "v1.1")

In [3]:
subset = dataset['test']

In [4]:
queries_info =[]
queries = []
corpus = []

In [7]:
subset

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 9650
})

In [11]:
import pandas as pd

pd.DataFrame(subset)

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Yes],"{'is_selected': [0, 0, 1, 0, 0, 0, 0], 'passag...",does human hair stop squirrels,0,description,[]
1,[Fossil fuels are basically the remains of ani...,"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0], '...",what are the benefits of fossil fuels,1,description,[]
2,[The apothem of a regular polygon is a line se...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0], '...",what is a apothem,2,description,[]
3,[$45 to $210. 2],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0], '...",average cost for custom canopy,3,numeric,[]
4,[It is the collection of physical elements tha...,"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",what is a hardware in a computer,4,description,[]
...,...,...,...,...,...,...
9645,[Can last 3-4 days in the fridge as long as it...,"{'is_selected': [0, 0, 0, 1, 0, 0, 0], 'passag...",how long can you keep bacon in the fridge,9647,numeric,[]
9646,[Body mass index (BMI) the weight in kilograms...,"{'is_selected': [0, 0, 0, 0, 0, 0, 1, 0, 0], '...",what is growth bmi mean in medical terms,9648,description,[]
9647,[Yes],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0], '...",can an llc apply for more than one dba,9649,description,[]
9648,['Bisque' is a shade of White that is 23% satu...,"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0], 'pas...",bisque color definition,9650,entity,[]


In [6]:
subset.shape

(9650, 6)

In [18]:
subset[5]['passages']

{'is_selected': [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 'passage_text': ['Like many other early information technologies, EDI was inspired by developments in military logistics. The complexity of the 1948 Berlin airlift required the development of concepts and methods to exchange, sometimes over a 300 baud teletype modem, vast quantities of data and information about transported goods. In EDI terminology inbound and outbound refer to the direction of transmission of an EDI document in relation to a particular system, not the direction of merchandise, money or other things represented by the document.',
  'Each term in the definition is significant to the understanding of how EDI works: Computer-to-computer – EDI replaces postal mail, fax and email. While email is also an electronic approach, the documents exchanged via email must still be handled by people rather than computers. The most commonly used EDI Transaction Sets in the Transportation Industry are: 1  204 - Motor Carrier Load Tender:

In [23]:
for sample in subset:
    query_type = sample['query_type']
    if query_type != 'entity':
        continue
    query_id = sample ['query_id']
    query_str = sample ['query']
    passages_dict = sample ['passages']
    is_selected_lst = passages_dict ['is_selected']
    passage_text_lst = passages_dict ['passage_text']

    query_info = {
        'query_id': query_id,
        'query': query_str,
        'relevant_doc': []
    }

    current_len_corpus = len(corpus)
    for idx in range(len(is_selected_lst)):
        if is_selected_lst[idx] == 1:
            doc_idx = current_len_corpus + idx
            query_info['relevant_doc'].append(doc_idx)
    if query_info['relevant_doc'] == []:
        continue

    queries.append(query_str)
    queries_info.append(query_info)
    corpus += passage_text_lst

In [24]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
english_stopwords = stopwords.words('english')
remove_chars = string.punctuation
stemmer = PorterStemmer()

def text_normalize(text):
    text = text.lower()
    for char in remove_chars:
        text = text.replace(char, "")
    text = " ".join([word for word in text.split() if word not in english_stopwords])
    text = " ".join([stemmer.stem(word) for word in text.split()])

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def create_dictionary(corpus):
    dictionary = []
    for doc in corpus:
        normalized_doc = text_normalize(doc)
        tokens = normalized_doc.split()
        for token in tokens:
            if token not in dictionary:
                dictionary.append(token)
    return dictionary

In [37]:
def vectorize(text, dictionary):
    word_count_dict = {word: 0 for word in dictionary}
    tokens = text.split()
    for token in tokens:
        try:
            word_count_dict[token] += 1
        except:
            pass
    vector = list(word_count_dict.values())
    return vector

In [39]:
def create_doc_term_matrix(corpus, dictionary):
    doc_term_matrix = {}
    for idx, doc in enumerate(corpus):
        normalized_doc = text_normalize(doc)
        vector = vectorize(normalized_doc, dictionary)
        doc_term_matrix[(doc, idx)] = vector
    return doc_term_matrix

In [28]:
from scipy import spatial

def similarity(a, b):
    return 1 - spatial.distance.cosine(a, b)

In [29]:
def ranking(query, dictionary, doc_term_matrix):
    normalized_query = text_normalize(query)
    query_vec = vectorize(normalized_query, dictionary)
    scores = []
    for doc_info, doc_vec in doc_term_matrix.items():
        sim = similarity(query_vec, doc_vec)
        scores.append((sim, doc_info))
    scores.sort(reverse=True)
    return scores

In [40]:
dictionary = create_dictionary(corpus)
doc_term_matrix = create_doc_term_matrix(corpus, dictionary)

In [41]:
query_lst = ['what is the official language in Fiji']
top_k = 10

for query in query_lst:
    scores = ranking(query, dictionary, doc_term_matrix)
    print (f'Query : { query }')
    print ('=== Relevant docs ===')
    for idx in range(top_k):
        doc_score = scores[idx][0]
        doc_content = scores[idx][1]
        print (f'Top { idx + 1}; Score : { doc_score :.4f}')
        print ( doc_content )
        print ('\n')

Query : what is the official language in Fiji
=== Relevant docs ===
Top 1; Score : 0.6556
('The official languages in Fiji are Fijian and English. A dialect of Hindustani is also widely spoken among Indo-Fijians.  _________________________________________   T … he official and everyday language of Fiji is English. Fijian and Fiji-Hindi are second languages in the island nation.', 6388)


Top 2; Score : 0.6556
('The official languages in Fiji are Fijian and English. A dialect of Hindustani is also widely spoken among Indo-Fijians.  _________________________________________   T … he official and everyday language of Fiji is English. Fijian and Fiji-Hindi are second languages in the island nation.', 6386)


Top 3; Score : 0.5715
('The official languages. Fiji’s 1997 Constitution established Fijian as one of the official languages of the country. Fijian is an Austronesian language, a grouping that includes thousands of other languages spanning the globe. The language is of the Malayo-Polyn